In [7]:
import os
import importlib
from utility import Pi_util

In [17]:
test_path: str = "/Volumes/EXTERNAL1/12SecondChunks_5hz_0NDF"

for file in os.listdir(test_path):
    if('.txt' in file):
        os.rename(os.path.join(test_path, file), os.path.join(test_path, file.replace('.txt', '_sunglasses.csv')))

AttributeError: module 'os' has no attribute 'move'

In [16]:
test_path: str = "/Volumes/EXTERNAL1/12SecondChunks_5hz_0NDF"

importlib.reload(Pi_util)
Pi_util.parse_chunks(test_path)

['burst0', 'burst1', 'burst2', 'burst3', 'burst4', 'burst5', 'burst6', 'burst7', 'burst8', 'burst9']
